In [1]:
import torch

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

print(device)

cuda


In [3]:
batch_size = 256
epochs = 30

## 이미지 전처리 작업

In [4]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder # 나만의 데이터셋을 불러오기. 계층적인 폴더구조 가져야됨

In [5]:
transform_base = transforms.Compose([transforms.Resize((64, 64)),
                                    transforms.ToTensor()])
# 각종 transforms들을 Compose로 한번에 묶어주기 가능

In [6]:
train_dataset = ImageFolder(root='./splitted/train/', transform = transform_base)
val_dataset = ImageFolder(root='./splitted/val/', transform = transform_base)

In [7]:
from torch.utils.data import DataLoader as DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
# num_workers를 통해 multi-process 가능

## 모델설계

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512) # 64*64 = 4096
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [10]:
model_base = Net().to(device)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [11]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [12]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1] # output 중 결과값 제일 큰거 출력
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [13]:
import time
import copy

In [14]:
def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs +1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        # 검증셋 결과가 가장 좋은 것으로 가중치 갱신    
        time_elapsed = time.time() - since
        print(f'------------epoch {epoch}------------')
        
        print('train loss: {:.4f}, accuracy: {:.2f}'.format(train_loss, train_acc))
        print('val loss: {:.4f}, accuracy: {:.2f}'.format(val_loss, val_acc))
        print("Completed in {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed%60))
        
    model.load_state_dict(best_model_wts)
    
    return model

In [15]:
base = train_baseline(model_base, train_loader, val_loader, optimizer, epochs)

------------epoch 1------------
train loss: 1.5550, accuracy: 54.86
val loss: 1.5866, accuracy: 54.01
Completed in 1m 18s
------------epoch 2------------
train loss: 0.9650, accuracy: 70.50
val loss: 1.0122, accuracy: 68.81
Completed in 1m 17s
------------epoch 3------------
train loss: 0.7233, accuracy: 77.08
val loss: 0.7959, accuracy: 74.85
Completed in 1m 17s
------------epoch 4------------
train loss: 0.6121, accuracy: 80.43
val loss: 0.6881, accuracy: 78.25
Completed in 1m 19s
------------epoch 5------------
train loss: 0.5190, accuracy: 83.30
val loss: 0.6103, accuracy: 80.62
Completed in 1m 19s
------------epoch 6------------
train loss: 0.3905, accuracy: 87.64
val loss: 0.4808, accuracy: 84.28
Completed in 1m 17s
------------epoch 7------------
train loss: 0.4658, accuracy: 84.66
val loss: 0.5634, accuracy: 81.39
Completed in 1m 16s
------------epoch 8------------
train loss: 0.2860, accuracy: 90.93
val loss: 0.3905, accuracy: 86.98
Completed in 1m 16s
------------epoch 9-----

In [16]:
torch.save(base.state_dict(), 'baseline.pt')

## 모델 불러와서 성능평가하기

In [21]:
baseline = Net().to(device)
baseline.load_state_dict(torch.load('baseline.pt'))
baseline.eval()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=33, bias=True)
)

In [22]:
test_base = ImageFolder(root='./splitted/test/', transform=transform_base)

In [23]:
test_loader_base = torch.utils.data.DataLoader(test_base,
                                              batch_size=batch_size,shuffle = True, num_workers=4)

In [24]:
test_loss, test_accuracy = evaluate(baseline, test_loader_base)
print('baseline test acc:', test_accuracy)

baseline test acc: 93.01539616973338
